# Imports

In [1]:
import os
import pathlib
import random
from math import sqrt
import ujson

import pandas as pd
import plotly.express as px

PATH = pathlib.Path.cwd() / "src"

# set current working directory to the path
os.chdir(PATH)
print(f"Current path: {PATH}")

Current path: /Users/eliebrosset/Documents/GitHub/cymoo/src


In [2]:
from MOO import MOO
from optimizers.NSGA2 import NSGA2
from optimizers.NSRA import NSRA
from optimizers.NSWGE import NSWGE
from problems.multitask_routing.Network import Network
from problems.multitask_routing.Task import Task
from problems.Solution import Solution

# Defining the parameters

## Layers

In [3]:
# PROBLEM DEFINITION
paramsLayerOne = {
    "unit": {
        "tag": "DEVICE",
        "computingSpeed": lambda: random.randint(5, 10),
        "positionX": 0,
        "positionY": 0,
        "throughput": 0.5,
        "pollution": lambda: round(random.random(), 2),
        "cost": lambda: round(random.random(), 2),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 1,
        "flowRate": lambda: 1,
    },
    "numberNewUnits": 10,
}

paramsLayerTwo = {
    "unit": {
        "tag": "FOG",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(5, 10),
        "positionX": lambda x: x.positionX + random.randint(-2, 2),
        "positionY": lambda x: x.positionY + random.randint(-2, 2),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 2
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 20,
}

paramsLayerThree = {
    "unit": {
        "tag": "CLOUD",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(50, 100),
        "positionX": lambda x: x.positionX + random.randint(-5, 5),
        "positionY": lambda x: x.positionY + random.randint(-5, 5),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 3
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 10,
}

## Tasks

In [4]:
tasks = tuple(
    Task(random.randint(1, 10000), random.randint(1, 10000)) for _ in range(3)
)

## Network

In [5]:
optimUnits = {
    "processingTime": "s",
    "cost": "€",
    "pollution": "g",
}
optimDirections = {
    "processingTime": "min",
    "cost": "min",
    "pollution": "min",
}
problem = Network(
    "DEVICE",
    tasks=tasks,
    optimDirections=optimDirections,
    minDepth=1,
    maxDepth=15,
    mutationRate=0.1,
    layers=[paramsLayerOne, paramsLayerTwo, paramsLayerThree],
)

## Optimization

In [6]:
nIterations = 10
nSolutions = 1000
seed = 10

# Optimization

## Training

In [7]:
moo = MOO(problem)

nsga2_paretos = moo.optimize(
    NSGA2,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    seed=seed,
    ratioKept=0.5,
)
nsra_paretos = moo.optimize(
    NSRA,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    seed=seed,
    ratioKept=0.5,
)
nswge_paretos = moo.optimize(
    NSWGE,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    seed=seed,
)

Optimizing with NSGA2...


100%|██████████| 10/10 [00:06<00:00,  1.51it/s]


Displaying pareto solutions...
{'processingTime': 16948.11111111111, 'cost': 14761.401904761904, 'pollution': 8664.989047619047}
{'processingTime': 16948.11111111111, 'cost': 14761.401904761904, 'pollution': 8664.989047619047}

Optimizing with NSRA...


100%|██████████| 10/10 [00:06<00:00,  1.50it/s]


Displaying pareto solutions...
{'processingTime': 93672.95842712474, 'cost': 77649.75824, 'pollution': 76902.500574}
{'processingTime': 43461.909032376745, 'cost': 58842.58509139044, 'pollution': 90579.59250877574}
{'processingTime': 48188.90527302178, 'cost': 90542.85371033152, 'pollution': 76586.86203437348}
{'processingTime': 72230.16083602913, 'cost': 112781.12317083502, 'pollution': 73959.74003383196}

Optimizing with NSWGE...


  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'dict' object has no attribute 'unit'

## Evaluation

In [ ]:
MOO.relative_efficiency(
    nsra_paretos, nsga2_paretos, Solution.optimDirections, verbose=True
)

# Results

In [ ]:
def generate_violin_plot(df, title):
    fig = px.violin(
        df,
        x="Epoch",
        y="Value",
        box=True,
        points=False,
        hover_data=df.columns,
        template="plotly_white",
        color_discrete_sequence=["#1f77b4"],
    )
    fig.update_layout(
        title=title,
        xaxis_title="Epoch",
        yaxis_title="Value",
        font=dict(
            family="Times New Roman",
            size=12,
        ),
    )
    fig.show()


def generate_random_values(epochs: int = 20):
    values = []
    for i in range(1, epochs + 1):
        for j in range(0, 10):
            values.append([i, random.randint(1, 100)])
    return values


def plot_violin(epochs):
    values = generate_random_values(epochs)
    df = pd.DataFrame(values, columns=["Epoch", "Value"])
    df = df[df["Epoch"] % (epochs // 10) == 0]
    generate_violin_plot(
        df, "Violin plot of the pareto solutions for each epoch (10 epochs)"
    )

plot_violin(50)

In [ ]:
objectives = optimDirections.keys()
objectives

dict_keys(['processingTime', 'cost', 'pollution'])

In [ ]:
# load pareto data from json
for file in os.listdir(PATH.parent / "paretos"):
    if file.endswith(".json"):
        # load file as dict
        print(file)
        with open(PATH.parent / "paretos" / file, "r") as f:
            data = ujson.load(f)
        for objective in objectives:
            # unstack values
            df = (
                pd.DataFrame.from_dict(data[objective], orient="index")
                .stack()
                .reset_index()
                .rename(columns={"level_0": "Epoch", 0: "Value"})
                .drop(columns="level_1")
            )
            # generate plot
            generate_violin_plot(
                df,
                f"Violin plot of the pareto solutions for each epoch (10 epochs) for the {objective} objective",
            )


NSGA2.json


NSRA.json
